<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/Frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import shutil
from google.colab import files
import sys
import tempfile
import urllib.request

# --- 안정적인 Face Swap 설치 및 설정 ---
print("🔥 Face Swap 라이브러리 설치 중...")

# 필요 라이브러리 설치
!pip install opencv-python-headless -q
!pip install insightface==0.7.3 -q
!pip install onnxruntime -q

print("✅ 라이브러리 설치 완료!")

# --- Face Swap 모델 설정 ---
device = 'cpu'
print(f"🚀 사용 장치: {device}")

def download_model_manually():
    """수동으로 모델 다운로드"""
    try:
        model_dir = '/root/.insightface/models'
        os.makedirs(model_dir, exist_ok=True)

        # 대안 URL들
        model_urls = [
            'https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx',
            'https://github.com/facefusion/facefusion-assets/releases/download/models/inswapper_128.onnx',
        ]

        model_path = os.path.join(model_dir, 'inswapper_128.onnx')

        for url in model_urls:
            try:
                print(f"📥 모델 다운로드 시도: {url}")
                urllib.request.urlretrieve(url, model_path)
                if os.path.exists(model_path) and os.path.getsize(model_path) > 1000000:  # 1MB 이상
                    print("✅ 모델 다운로드 성공!")
                    return model_path
            except Exception as e:
                print(f"❌ 다운로드 실패: {e}")
                continue

        return None
    except Exception as e:
        print(f"❌ 수동 다운로드 실패: {e}")
        return None

try:
    import insightface
    from insightface.app import FaceAnalysis

    # Face Analysis 모델 초기화 (CPU 사용)
    app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
    app.prepare(ctx_id=-1, det_size=(640, 640))

    # Face Swapper 모델 로드 시도
    swapper = None
    try:
        swapper = insightface.model_zoo.get_model('inswapper_128.onnx', download=True, download_zip=True)
        print("✅ InsightFace Swapper 로드 성공!")
    except:
        print("⚠️ 기본 다운로드 실패, 수동 다운로드 시도...")
        model_path = download_model_manually()
        if model_path:
            try:
                swapper = insightface.model_zoo.get_model(model_path)
                print("✅ 수동 다운로드 모델 로드 성공!")
            except Exception as e:
                print(f"❌ 수동 모델 로드 실패: {e}")
                swapper = None

    if swapper:
        MODEL_LOADED = "INSIGHTFACE"
        print("✅ 고품질 InsightFace 모델 준비 완료!")
    else:
        MODEL_LOADED = "BASIC"
        print("⚠️ 기본 방법으로 전환합니다.")

except Exception as e:
    print(f"❌ InsightFace 로드 실패: {e}")
    MODEL_LOADED = "BASIC"

# 기본 얼굴 감지기 (백업용)
if MODEL_LOADED == "BASIC":
    try:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        print("✅ 기본 얼굴 감지기 로드 완료!")
    except Exception as e:
        print(f"❌ 모든 모델 로드 실패: {e}")
        MODEL_LOADED = False

def get_face_landmarks_basic(img):
    """기본적인 얼굴 랜드마크 추출 (개선됨)"""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    if len(faces) == 0:
        return None

    # 가장 큰 얼굴 선택
    face = max(faces, key=lambda x: x[2] * x[3])
    x, y, w, h = face

    # 얼굴 영역에서 눈 감지
    face_gray = gray[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(face_gray, scaleFactor=1.1, minNeighbors=3)

    # 기본 랜드마크 포인트
    if len(eyes) >= 2:
        # 실제 눈 위치 사용
        eye1, eye2 = sorted(eyes, key=lambda e: e[0])[:2]  # x좌표로 정렬
        left_eye = (x + eye1[0] + eye1[2]//2, y + eye1[1] + eye1[3]//2)
        right_eye = (x + eye2[0] + eye2[2]//2, y + eye2[1] + eye2[3]//2)
    else:
        # 추정 위치
        left_eye = (x + w//4, y + h//3)
        right_eye = (x + 3*w//4, y + h//3)

    landmarks = {
        'left_eye': left_eye,
        'right_eye': right_eye,
        'nose': (x + w//2, y + h//2),
        'mouth': (x + w//2, y + 2*h//3)
    }

    return landmarks, (x, y, w, h)

def create_smooth_mask(shape, center, radius):
    """부드러운 마스크 생성"""
    h, w = shape[:2]
    y, x = np.ogrid[:h, :w]
    mask = np.zeros((h, w), dtype=np.float32)

    # 중심에서의 거리 계산
    dist_from_center = np.sqrt((x - center[0])**2 + (y - center[1])**2)

    # 부드러운 그라디언트 마스크
    mask = np.where(dist_from_center <= radius * 0.8, 1.0,
                   np.where(dist_from_center <= radius,
                           1.0 - (dist_from_center - radius * 0.8) / (radius * 0.2), 0.0))

    return mask

def process_single_frame_insightface(source_face, target_frame):
    """InsightFace를 사용한 고품질 얼굴 교체"""
    try:
        target_faces = app.get(target_frame)

        if len(target_faces) == 0:
            return target_frame

        # 가장 큰 얼굴 선택
        target_face = max(target_faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))

        # 얼굴 교체 수행
        result = swapper.get(target_frame, target_face, source_face, paste_back=True)

        return result

    except Exception as e:
        print(f"InsightFace 처리 오류: {e}")
        return target_frame

def process_single_frame_basic(source_img, source_face_region, target_frame):
    """개선된 기본 얼굴 교체"""
    try:
        target_landmarks = get_face_landmarks_basic(target_frame)

        if target_landmarks is None:
            return target_frame

        landmarks, face_rect = target_landmarks
        tx, ty, tw, th = face_rect

        # 소스 얼굴 추출 및 전처리
        sx, sy, sw, sh = source_face_region
        source_face = source_img[sy:sy+sh, sx:sx+sw].copy()

        # 타겟 크기에 맞게 리사이즈
        resized_face = cv2.resize(source_face, (tw, th), interpolation=cv2.INTER_CUBIC)

        # 색상 보정 (히스토그램 매칭)
        target_region = target_frame[ty:ty+th, tx:tx+tw]

        # 각 채널별로 히스토그램 매칭
        for i in range(3):
            # 간단한 색상 매칭
            target_mean = np.mean(target_region[:, :, i])
            source_mean = np.mean(resized_face[:, :, i])

            if source_mean > 0:
                resized_face[:, :, i] = np.clip(
                    resized_face[:, :, i] * (target_mean / source_mean), 0, 255
                ).astype(np.uint8)

        # 타원형 마스크 생성
        center = (tw//2, th//2)
        radius = min(tw, th) // 2
        mask = create_smooth_mask((th, tw), center, radius)

        # 3채널로 확장
        mask_3d = np.stack([mask] * 3, axis=-1)

        # 부드러운 블렌딩
        result = target_frame.copy()
        blended = target_region * (1 - mask_3d) + resized_face * mask_3d
        result[ty:ty+th, tx:tx+tw] = blended.astype(np.uint8)

        return result

    except Exception as e:
        print(f"기본 처리 오류: {e}")
        return target_frame

def extract_frames(video_path, output_dir):
    """비디오에서 프레임 추출"""
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frames_dir = os.path.join(output_dir, 'frames')
    os.makedirs(frames_dir, exist_ok=True)

    print(f"📹 비디오 정보: {width}x{height}, {fps:.1f}fps, {frame_count}프레임")

    frame_paths = []
    for frame_idx in tqdm(range(frame_count), desc="🎞️ 프레임 추출"):
        ret, frame = cap.read()
        if not ret:
            break

        frame_path = os.path.join(frames_dir, f"frame_{frame_idx:06d}.jpg")
        cv2.imwrite(frame_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
        frame_paths.append(frame_path)

    cap.release()
    return frame_paths, fps, width, height

def process_frames(source_data, frame_paths, output_dir, method):
    """프레임 처리 (방법에 따라 분기)"""
    processed_dir = os.path.join(output_dir, 'processed')
    os.makedirs(processed_dir, exist_ok=True)

    processed_paths = []
    success_count = 0

    desc = f"🎭 Face Swapping ({method})"

    for i, frame_path in enumerate(tqdm(frame_paths, desc=desc)):
        try:
            # 프레임 로드
            frame = cv2.imread(frame_path)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # 방법에 따라 처리
            if method == "INSIGHTFACE":
                result_frame = process_single_frame_insightface(source_data, frame_rgb)
            else:  # BASIC
                source_img, source_face_region = source_data
                result_frame = process_single_frame_basic(source_img, source_face_region, frame_rgb)

            # 결과 저장
            result_bgr = cv2.cvtColor(result_frame, cv2.COLOR_RGB2BGR)
            output_path = os.path.join(processed_dir, f"processed_{i:06d}.jpg")
            cv2.imwrite(output_path, result_bgr, [cv2.IMWRITE_JPEG_QUALITY, 95])
            processed_paths.append(output_path)

            # 성공 여부 확인
            if not np.array_equal(frame_rgb, result_frame):
                success_count += 1

        except Exception as e:
            print(f"프레임 {i} 처리 실패: {e}")
            # 실패시 원본 프레임 복사
            output_path = os.path.join(processed_dir, f"processed_{i:06d}.jpg")
            shutil.copy2(frame_path, output_path)
            processed_paths.append(output_path)

    print(f"✅ 프레임 처리 완료: {success_count}/{len(frame_paths)} 성공")
    return processed_paths, success_count

def create_video_from_frames(frame_paths, output_path, fps, width, height):
    """처리된 프레임들로 비디오 생성"""
    # H.264 코덱 사용 (더 호환성 좋음)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame_path in tqdm(frame_paths, desc="🎬 비디오 생성"):
        frame = cv2.imread(frame_path)
        if frame is not None:
            out.write(frame)

    out.release()
    print(f"✅ 비디오 생성 완료: {output_path}")

# --- 파일 업로드 ---
print("\n📁 파일 업로드")

print("🤳 본인 얼굴 사진을 업로드하세요:")
source_files = files.upload()
if source_files:
    source_path = list(source_files.keys())[0]
    print(f"✅ 소스 이미지: {source_path}")
else:
    print("❌ 소스 이미지 업로드 실패")
    exit()

print("\n🎬 대상 비디오를 업로드하세요:")
target_files = files.upload()
if target_files:
    target_path = list(target_files.keys())[0]
    print(f"✅ 타겟 비디오: {target_path}")
else:
    print("❌ 타겟 비디오 업로드 실패")
    exit()

# 모델이 로드되지 않았다면 종료
if not MODEL_LOADED:
    print("❌ 모든 모델 로드에 실패했습니다.")
    exit()

# --- 소스 이미지 처리 ---
print("\n🔍 소스 얼굴 분석 중...")

source_img = cv2.imread(source_path)
if source_img is None:
    print("❌ 소스 이미지를 읽을 수 없습니다.")
    exit()

source_img = cv2.cvtColor(source_img, cv2.COLOR_BGR2RGB)

# 변수 초기화
source_data = None

if MODEL_LOADED == "INSIGHTFACE":
    try:
        # InsightFace로 소스 얼굴 감지
        source_faces = app.get(source_img)
        if len(source_faces) == 0:
            print("⚠️ InsightFace로 얼굴을 찾을 수 없어 기본 방법으로 전환합니다")
            MODEL_LOADED = "BASIC"
        else:
            # 가장 큰 얼굴 선택
            source_face = max(source_faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
            source_data = source_face
            print(f"✅ 소스 얼굴 감지 성공! (InsightFace 고품질)")

    except Exception as e:
        print(f"⚠️ InsightFace 얼굴 감지 실패: {e}")
        print("기본 방법으로 전환합니다...")
        MODEL_LOADED = "BASIC"

# 기본 방법 사용 (InsightFace 실패 시 또는 처음부터 기본 방법)
if MODEL_LOADED == "BASIC" or source_data is None:
    try:
        # 기본 방법으로 소스 얼굴 감지
        source_landmarks = get_face_landmarks_basic(source_img)
        if source_landmarks is None:
            print("❌ 소스 이미지에서 얼굴을 찾을 수 없습니다")
            exit()

        landmarks, source_face_region = source_landmarks
        source_data = (source_img, source_face_region)
        MODEL_LOADED = "BASIC"
        print(f"✅ 소스 얼굴 감지 성공! (기본 방법)")

    except Exception as e:
        print(f"❌ 모든 얼굴 감지 방법 실패: {e}")
        exit()

# 최종 확인
if source_data is None:
    print("❌ 소스 얼굴 데이터 준비 실패")
    exit()

# --- 메인 처리 과정 ---
print("\n🎥 비디오 처리 시작...")

# 출력 디렉토리 생성
output_dir = '/content/FaceSwap_output'
os.makedirs(output_dir, exist_ok=True)

# Step 1: 비디오에서 프레임 추출
print("1️⃣ 프레임 추출 중...")
frame_paths, fps, width, height = extract_frames(target_path, output_dir)

# Step 2: 각 프레임에서 얼굴 교체
print("2️⃣ 프레임별 얼굴 교체 중...")
processed_paths, success_count = process_frames(source_data, frame_paths, output_dir, MODEL_LOADED)

# Step 3: 처리된 프레임들로 비디오 생성
print("3️⃣ 최종 비디오 생성 중...")
final_video_path = os.path.join(output_dir, 'final_faceswap_video.mp4')
create_video_from_frames(processed_paths, final_video_path, fps, width, height)

# --- 결과 처리 ---
print(f"\n🎉 처리 완료!")
print(f"📊 성공률: {success_count}/{len(frame_paths)} ({success_count/len(frame_paths)*100:.1f}%)")
print(f"🔧 사용된 방법: {'InsightFace (고품질)' if MODEL_LOADED == 'INSIGHTFACE' else '개선된 기본 방법'}")

if os.path.exists(final_video_path) and os.path.getsize(final_video_path) > 0:
    print("📥 결과 다운로드 중...")
    files.download(final_video_path)

    # 미리보기 생성
    try:
        cap = cv2.VideoCapture(final_video_path)
        ret, preview_frame = cap.read()
        if ret:
            preview_path = os.path.join(output_dir, 'preview.jpg')
            cv2.imwrite(preview_path, preview_frame)
            files.download(preview_path)
            print("🖼️ 미리보기 이미지도 생성됨")
        cap.release()
    except:
        pass

    # 품질 확인용 샘플 프레임들
    try:
        sample_indices = [0, len(processed_paths)//4, len(processed_paths)//2, len(processed_paths)*3//4, -1]
        for i, idx in enumerate(sample_indices):
            if 0 <= idx < len(processed_paths):
                sample_path = processed_paths[idx]
                sample_name = f"sample_frame_{i+1}.jpg"
                sample_output = os.path.join(output_dir, sample_name)
                shutil.copy2(sample_path, sample_output)
                files.download(sample_output)
        print("📸 샘플 프레임들도 다운로드됨 (품질 확인용)")
    except:
        pass

else:
    print("❌ 비디오 생성 실패")

# --- 정리 ---
print("\n🧹 임시 파일 정리...")
try:
    # 프레임 파일들 정리 (용량 절약)
    shutil.rmtree(os.path.join(output_dir, 'frames'), ignore_errors=True)
    shutil.rmtree(os.path.join(output_dir, 'processed'), ignore_errors=True)

    # 업로드된 파일 정리
    os.remove(source_path)
    os.remove(target_path)
    print("✅ 임시 파일 정리 완료")
except Exception as e:
    print(f"정리 중 오류: {e}")

print("\n🎭 Face Swap 완료!")
print("⭐ 프레임별 처리로 안정적인 결과를 확인해보세요!")
if MODEL_LOADED == "BASIC":
    print("💡 더 좋은 품질을 원한다면 GPU 환경에서 실행하거나 고급 모델을 사용해보세요!")
print("⚠️ 윤리적으로 사용해주세요!")

🔥 Face Swap 라이브러리 설치 중...
✅ 라이브러리 설치 완료!
🚀 사용 장치: cpu


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
inswapper-shape: [1, 3, 128, 128]
✅ InsightFace Swapper 로드 성공!
✅ 고품질 InsightFace 모델 준비 완료!

📁 파일 업로드
🤳 본인 얼굴 사진을 업로드하세요:


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📸 샘플 프레임들도 다운로드됨 (품질 확인용)

🧹 임시 파일 정리...
✅ 임시 파일 정리 완료

🎭 Face Swap 완료!
⭐ 프레임별 처리로 안정적인 결과를 확인해보세요!
💡 더 좋은 품질을 원한다면 GPU 환경에서 실행하거나 고급 모델을 사용해보세요!
⚠️ 윤리적으로 사용해주세요!
